<div align="center">
<img src="https://poorit.in/image.png" alt="Poorit" width="40" style="vertical-align: middle;"> <b>AI SYSTEMS ENGINEERING 1</b>

## Unit 2: Building User Interfaces with Gradio

**CV Raman Global University, Bhubaneswar**  
*AI Center of Excellence*

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Poorit-Technologies/cvraman-coe/blob/main/courses-contents/ai-systems-engineering-1/unit-2/01-ai-systems-engineering-1-unit2-gradio-ui-basics.ipynb)

</div>

---

### What You'll Learn

In this notebook, you will:

1. **Build user interfaces with Gradio** - the simplest way to create ML demos
2. **Create streaming LLM interfaces** for better user experience
3. **Add model selection dropdowns** to switch between providers
4. **Build a company pamphlet generator UI** as a practical project

**Duration:** ~1.5 hours

---

## 1. Environment Setup

In [4]:
# Install required packages
!pip install -q openai gradio requests beautifulsoup4

In [5]:
import os
from getpass import getpass
from openai import OpenAI
import gradio as gr
import requests
from bs4 import BeautifulSoup

In [6]:
# Configure API keys
openai_api_key = getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = openai_api_key

# Optional: Google API key for Gemini
google_api_key = getpass("Enter your Google API Key (or press Enter to skip): ")

Enter your OpenAI API Key: ··········
Enter your Google API Key (or press Enter to skip): ··········


In [7]:
# Initialize clients
openai_client = OpenAI(api_key=openai_api_key)

GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = OpenAI(api_key=google_api_key, base_url=GEMINI_URL) if google_api_key else None

MODEL = "gpt-4o-mini"

---

## 2. Your First Gradio Interface

Gradio makes it incredibly simple to create web UIs for Python functions.

The basic pattern is:
```python
gr.Interface(fn=your_function, inputs="textbox", outputs="textbox").launch()
```

In [8]:
# A simple function to demonstrate
def shout(text):
    return text.upper()

In [9]:
# Create a Gradio interface
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b67df8ee8f1d2adb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## 3. Connecting to an LLM

Let's wrap our LLM call in a function and connect it to Gradio.

In [10]:
system_message = "You are a helpful assistant that responds in markdown."

def message_gpt(prompt):
    """Send a message to GPT and return the response."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = openai_client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [11]:
# Create a more polished interface
message_input = gr.Textbox(label="Your message:", lines=5)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT Assistant",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain machine learning in simple terms",
        "What is the capital of India?"
    ],
    flagging_mode="never"
)
view.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94fc04109e8c315687.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## 4. Streaming Responses

Streaming shows the response as it's generated, providing a much better user experience.

We use Python **generators** (the `yield` keyword) for this.

In [ ]:
def stream_gpt(prompt):
    """Stream a response from GPT."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = openai_client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# Streaming interface
message_input = gr.Textbox(label="Your message:", lines=5)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT Assistant (Streaming)",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a beginner",
        "Write a short poem about coding"
    ],
    flagging_mode="never"
)
view.launch()

---

## 5. Model Selection Dropdown

Let's add the ability to choose between different models.

In [ ]:
def stream_gemini(prompt):
    """Stream a response from Gemini."""
    if not gemini_client:
        yield "Gemini API key not configured."
        return

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = gemini_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_model(prompt, model):
    """Stream response from selected model."""
    if model == "GPT":
        yield from stream_gpt(prompt)
    elif model == "Gemini":
        yield from stream_gemini(prompt)
    else:
        yield "Unknown model selected."

In [ ]:
# Interface with model selection
message_input = gr.Textbox(label="Your message:", lines=5)
model_selector = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="Multi-Model Assistant",
    inputs=[message_input, model_selector],
    outputs=[message_output],
    examples=[
        ["What is deep learning?", "GPT"],
        ["Explain neural networks", "Gemini"]
    ],
    flagging_mode="never"
)
view.launch()

---

## 6. Project: Company Pamphlet Generator

Let's build a practical application that scrapes a company website and generates a pamphlet.

In [ ]:
# Web scraping utility
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
}

def fetch_website_contents(url, max_chars=2000):
    """Fetch and return the text content of a website."""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""

        return (title + "\n\n" + text)[:max_chars]
    except Exception as e:
        return f"Error fetching website: {str(e)}"

In [ ]:
# Update system message for pamphlet generation
pamphlet_system = """
You are an assistant that analyzes company website content
and creates a short pamphlet for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

def stream_pamphlet(company_name, url, model):
    """Generate a company pamphlet with streaming."""
    yield ""

    website_content = fetch_website_contents(url)
    prompt = f"Please generate a company pamphlet for {company_name}. Here is their landing page:\n{website_content}"

    messages = [
        {"role": "system", "content": pamphlet_system},
        {"role": "user", "content": prompt}
    ]

    if model == "GPT":
        client = openai_client
        model_name = MODEL
    else:
        if not gemini_client:
            yield "Gemini API key not configured."
            return
        client = gemini_client
        model_name = "gemini-1.5-flash"

    stream = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# Pamphlet generator interface
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL (include https://):")
model_selector = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
pamphlet_output = gr.Markdown(label="Generated Pamphlet:")

view = gr.Interface(
    fn=stream_pamphlet,
    title="Company Pamphlet Generator",
    inputs=[name_input, url_input, model_selector],
    outputs=[pamphlet_output],
    examples=[
        ["Anthropic", "https://anthropic.com", "GPT"],
        ["Hugging Face", "https://huggingface.co", "Gemini"]
    ],
    flagging_mode="never"
)
view.launch()

---

## 7. Exercise: Build a Product Description Generator

Create a Gradio interface that takes a product URL and generates marketing copy.

In [ ]:
# Exercise: Create a product description generator
# 1. Define a system prompt for marketing copy
# 2. Create a streaming function that fetches product page and generates description
# 3. Build a Gradio interface

# Your implementation here
pass

---

## Key Takeaways

1. **Gradio makes UI creation simple** - just wrap your function with `gr.Interface`

2. **Streaming improves UX** - use generators (`yield`) for real-time output

3. **Dropdowns add flexibility** - easily switch between models or options

4. **Combine scraping + LLM** - powerful pattern for content generation

### What's Next?

In the next notebook, we'll explore:
- Building conversational AI with memory
- ChatInterface for chatbot UIs
- System prompts for context

---

## Additional Resources

- [Gradio Documentation](https://www.gradio.app/guides/quickstart)
- [Gradio Components](https://www.gradio.app/docs/components)

---

**Course Information:**
- **Institution:** CV Raman Global University, Bhubaneswar
- **Program:** AI Center of Excellence
- **Course:** AI Systems Engineering 1
- **Developed by:** [Poorit Technologies](https://poorit.in) - *Transform Graduates into Industry-Ready Professionals*

---